LSM EXP 05_Aiffel
<br/>**5. 인공지능과 가위바위보 하기**

라이브러리
<br/>데이터 정보
<br/>데이터 전처리
<br/>모델 학습
* Drop-Out 적용 모델 학습
* Drop-Out 미적용 모델 학습

모델 테스트
* Drop-Out 적용 모델 테스트
* Drop-Out 미적용 모델 테스트

가위, 바위, 보가 너무 쉬울때!
* Drop-Out 적용 모델 테스트
* Drop-Out 미적용 모델 테스트
* 과적합을 방지하는 방법

결론
<br/>참고문헌


#라이브러리

In [37]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os

**tensorflow**는 구글이 개발한 오픈소스 소프트웨어 딥러닝 및 머신러닝 라이브러리이다. 수학 계산식과 데이터의 흐름을 노드와 엣지를 사용한 방향성 그래프, 데이터 플로우 그래프로 나타낸다.


**Keras**는 Tensorflow 위에서 동작하는 라이브러리이다.
<br/>사용자 친화적으로 개발된 Keras의 쉽다는 장점과
<br/>딥러닝 프로젝트에서 범용적으로 활용할 수 있는
<br/>Tensorflow의 장점을 통합할 수 있는 환경을 설정한다.

**numpy**는 array 단위로 벡터와 행렬을 계산한다.


**sklearn.model_selection**은 훈련 데이터와 테스트 데이터를 분리한다.

**matplotlib**은 다양한 데이터와 학습 모델을 시각화한다.

**os(Operating System)**는 운영체제에서 제공되는 여러 기능을 파이썬에서 수행한다. <br/>예를 들어, 파일 복사, 디렉터리 생성, 파일 목록을 구할 수 있다.

In [38]:
from PIL import Image
import glob

**PIL(Python Image Library)**는 다양한 이미지 파일 형식을 지원하는 작업 모듈이다. 다만, PIL의 지원이 2011년 중단되고, Pillow가 PIL의 후속 프로젝트로 나왔다.


**glob**는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다. 단, 조건에 정규식을 사용할 수 없으며 엑셀 등에서도 사용할 수 있는 '*'와 '?'같은 와일드카드만을 지원한다.

##데이터 정보

**rock_sissor_paper**

2021년 아이펠(Aiffel) 수강생이 촬영했던 가위, 바위, 보 이미지를 통합하여 만든 데이터셋이다.
<br/>포토윅스라는 프로그램을 이용하여 이미지를 편집했다.

이 데이터셋은 224X224 pixel의 6792개의 이미지로 구성되어 있다.
<br/>가위(2270개), 바위(2202개), 보(2320개)의 이미지는 가위바위보 게임의 동작을 나타낸다.


##데이터 전처리

[훈련/테스트 데이터 폴더 분리_주피터 노트북](https://github.com/minseok0809/Aiffel/blob/main/LSMExp/%5BE-05%5Dtraintestsplit.ipynb)

주피터 노트북에서 훈련/테스트 데이터 폴더 분리를 했다.
<br/>구글 드라이브를 연동하는 코랩에서의 폴더 분리를 하는 법을 알 수 없어서 주피터 노트북에서 대신했다.

splitfolders을 이용하여 train/val/test set를 7:0:3으로 분리했다.

In [39]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")

파일마다 모두 28x28 사이즈로 바꾼다.

**왜 28X28인가?**
<br/>'Why 28x28 pixel'라고 검색해보았으나 나오지 않는다.
<br/>784(28 x 28)개의 각 뉴런들은 각 픽셀의 밝기를 나타낸다고만 검색 결과가 나올 뿐이다.

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [110]:
image_dir_path1 = '/content/drive/MyDrive/LMS/rock_scissor_paper/train/scissor_train/'
resize_images(image_dir_path1)

1624  images to be resized.
1624  images resized.


가위 이미지를 28x28 사이즈로 바꾼다.

In [111]:
image_dir_path2 = '/content/drive/MyDrive/LMS/rock_scissor_paper/train/rock_train/'
resize_images(image_dir_path2)

1541  images to be resized.
1541  images resized.


바위 이미지를 28x28 사이즈로 바꾼다.

In [113]:
image_dir_path3 = '/content/drive/MyDrive/LMS/rock_scissor_paper/train/paper_train/'
resize_images(image_dir_path3)

1590  images to be resized.
1590  images resized.


보 이미지를 28x28 사이즈로 바꾼다.

In [114]:
def load_data(img_path, number_of_data):
    img_size = 28
    color = 3
    
    imgs = np.zeros(number_of_data * img_size * img_size * color, dtype = np.int32).reshape(number_of_data, img_size, img_size, color)
    labels = np.zeros(number_of_data, dtype = np.int32)

    idx = 0
    for file in glob.iglob(img_path + '/scissor_train/*.jpg'):
        img = np.array(Image.open(file), dtype = np.int32)
        imgs[idx, :, :, :] = img
        labels[idx] = 0
        idx = idx + 1

    for file in glob.iglob(img_path + '/rock_train/*.jpg'):
        img = np.array(Image.open(file), dtype = np.int32)
        imgs[idx, :, :, :] = img
        labels[idx] = 1
        idx = idx + 1       
    
    for file in glob.iglob(img_path + '/paper_train/*.jpg'):
        img = np.array(Image.open(file), dtype = np.int32)
        imgs[idx, :, :, :] = img
        labels[idx] = 2
        idx = idx + 1
        
    return imgs, labels

라벨을 가위(0), 바위(1), 보(2)로 한다.
<br/>이미지와 라벨을 담은 행렬을 생성한다.

for문을 실행하기 앞서 np.zeros와 reshape를 통해 행렬을 초기화한다.
<br/>**np.zeros**는 크기(number_of_data*img_size*img_size*color)만큼의 0으로만 채워진 1차원 벡터를 생성한다.

**reshape**은 크기(number_of_data,img_size,img_size,color)만큼의 3차원 텐서를 생성한다.

In [115]:
image_dir_path4 = '/content/drive/MyDrive/LMS/rock_scissor_paper/train/'
(x_train,y_train) = load_data(image_dir_path4, 4755)
x_train_norm = x_train/255.0

훈련 데이터 4755개, 테스트 데이터 2038개로 분리한 폴더에서 훈련 데이터를 불러온다.

255로 나누어 입력은 0~1 사이의 값으로 정규화한다.


In [116]:
print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

x_train shape: (4755, 28, 28, 3)
y_train shape: (4755,)


##모델 학습

Drop-Out 적용한 모델의 이름은 'model1'
<br/>Drop-Out 적용하지 않은 모델의 이름은 'model2'로 설정한다.

###Drop-Out 적용 모델 학습


In [136]:
n_channel_1 = 256
n_channel_2 = 512
n_dense = 512
n_class = 3
n_drop_rate = 0.3
n_train_epoch = 10

레이어의 개수, 분류 클래스의 개수, drop rate, 최적화의 학습단위(train epoch) 등 하이퍼파라미터 튜닝을 한다.

**Drop-out**은 서로 연결된 연결망(layer)에서 0부터 1 사이의 확률로 뉴런을 제거(drop)하는 기법이다.
<br/>Drop-out을 적용하여 상관관계가 강한 Feature를 제외하여
<br/>해당 Feature에만 출력값이 좌지우지되는 과대적합(overfitting)을 방지한다.

In [137]:
model1 =keras.models.Sequential()
model1.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model1.add(keras.layers.MaxPool2D(2,2))
model1.add(keras.layers.Dropout(n_drop_rate))
model1.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model1.add(keras.layers.MaxPooling2D((2,2)))
model1.add(keras.layers.Dropout(n_drop_rate))
model1.add(keras.layers.Flatten())
model1.add(keras.layers.Dense(n_dense, activation='relu'))
model1.add(keras.layers.Dense(n_class, activation='softmax'))

print('Model1 (Drop-Out 적용)에 추가된 Layer 개수: ', len(model1.layers))

Model1 (Drop-Out 적용)에 추가된 Layer 개수:  9


첫번째 레이어는 사이즈 3의 256개의 필터로 구성되어 있다. 이미지 형태는 28X28 크기이다.
<br/>**relu**는 활성화함수로 구성된다.
<br/>**2 x 2 max-pooling** 레이어를 가진다. 추상화된 형태를 오버피팅을 방지하는데 도움을 준다.
<br/>**Flatten**은 입력을 1차원으로 변환한다.
<br/>**Dropou**t은 오버피팅을 방지한다.
<br/>**Dense**는 최종적으로 지정된 Class로 분류한다.

In [138]:
model1.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 26, 26, 256)       7168      
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 13, 13, 256)      0         
 g2D)                                                            
                                                                 
 dropout_12 (Dropout)        (None, 13, 13, 256)       0         
                                                                 
 conv2d_19 (Conv2D)          (None, 11, 11, 512)       1180160   
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 5, 5, 512)        0         
 g2D)                                                            
                                                                 
 dropout_13 (Dropout)        (None, 5, 5, 512)        

In [139]:
model1.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model1.fit(x_train_norm, y_train, epochs= n_train_epoch)

Epoch 1/10
149/149 [==============================] - 2s 10ms/step - loss: 1.1114 - accuracy: 0.3516
Epoch 2/10
149/149 [==============================] - 1s 10ms/step - loss: 0.9955 - accuracy: 0.4927
Epoch 3/10
149/149 [==============================] - 1s 10ms/step - loss: 0.7277 - accuracy: 0.6738
Epoch 4/10
149/149 [==============================] - 1s 10ms/step - loss: 0.5363 - accuracy: 0.7676
Epoch 5/10
149/149 [==============================] - 1s 10ms/step - loss: 0.3568 - accuracy: 0.8559
Epoch 6/10
149/149 [==============================] - 1s 10ms/step - loss: 0.2471 - accuracy: 0.9110
Epoch 7/10
149/149 [==============================] - 1s 10ms/step - loss: 0.1913 - accuracy: 0.9365
Epoch 8/10
149/149 [==============================] - 1s 10ms/step - loss: 0.1340 - accuracy: 0.9535
Epoch 9/10
149/149 [==============================] - 1s 10ms/step - loss: 0.1113 - accuracy: 0.9617
Epoch 10/10
149/149 [==============================] - 1s 10ms/step - loss: 0.0882 - accura

하이퍼 파라미터 최적화 알고리즘으로 Adam을 사용한다.

[Adam]
<br/>모멘텀과 AdaGrad를 결합한다.
<br/>매개변수 공간을 효율적으로 탐색해주며
<br/>하이퍼파라미터의 '편향 보정'이 진행된다는 점이 Adam의 특징이다.

Dropout을 적용했을 때의 모델 성능이다.
<br/>loss가 0.0882일 때, 97.06%의 accuracy가 나온다.

###drop rate 미적용 모델 학습

In [140]:
n_channel_1 = 256
n_channel_2 = 512
n_channel_3 = 512
n_dense = 512
n_class = 3
n_train_epoch = 10

In [141]:
model2 =keras.models.Sequential()
model2.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model2.add(keras.layers.MaxPooling2D(2,2))
model2.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model2.add(keras.layers.MaxPooling2D((2,2)))
model2.add(keras.layers.Conv2D(n_channel_3, (3,3), activation='relu'))
model2.add(keras.layers.MaxPooling2D((2,2)))
model2.add(keras.layers.Flatten())
model2.add(keras.layers.Dense(n_dense, activation='relu'))
model2.add(keras.layers.Dense(n_class, activation='softmax'))

print('Model2 (Drop-Out 미적용)에 추가된 Layer 개수: ', len(model2.layers))

Model2 (Drop-Out 미적용)에 추가된 Layer 개수:  9


In [142]:
model2.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 26, 26, 256)       7168      
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 13, 13, 256)      0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 11, 11, 512)       1180160   
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 5, 5, 512)        0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 3, 3, 512)         2359808   
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 1, 1, 512)       

In [143]:
model2.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model2.fit(x_train_norm, y_train, epochs= n_train_epoch)

Epoch 1/10
149/149 [==============================] - 2s 12ms/step - loss: 1.0984 - accuracy: 0.3453
Epoch 2/10
149/149 [==============================] - 2s 12ms/step - loss: 0.8671 - accuracy: 0.5880
Epoch 3/10
149/149 [==============================] - 2s 12ms/step - loss: 0.4842 - accuracy: 0.7962
Epoch 4/10
149/149 [==============================] - 2s 12ms/step - loss: 0.2454 - accuracy: 0.9079
Epoch 5/10
149/149 [==============================] - 2s 12ms/step - loss: 0.1357 - accuracy: 0.9537
Epoch 6/10
149/149 [==============================] - 2s 12ms/step - loss: 0.0796 - accuracy: 0.9714
Epoch 7/10
149/149 [==============================] - 2s 12ms/step - loss: 0.1160 - accuracy: 0.9647
Epoch 8/10
149/149 [==============================] - 2s 12ms/step - loss: 0.0281 - accuracy: 0.9903
Epoch 9/10
149/149 [==============================] - 2s 12ms/step - loss: 0.0442 - accuracy: 0.9853
Epoch 10/10
149/149 [==============================] - 2s 12ms/step - loss: 0.0317 - accura

Dropout을 적용하지 않았을 때 모델 성능이 더 높다.
<br/>loss가 0.0317일 때, 99.05%의 accuracy가 나온다.

##모델 테스트

훈련 데이터를 전처리한다.

In [184]:
image_dir_path5 = '/content/drive/MyDrive/LMS/rock_scissor_paper/test/scissor_test/'
resize_images(image_dir_path5)

696  images to be resized.
696  images resized.


In [185]:
image_dir_path6 = '/content/drive/MyDrive/LMS/rock_scissor_paper/test/rock_test/'
resize_images(image_dir_path6)

661  images to be resized.
661  images resized.


In [186]:
image_dir_path7 = '/content/drive/MyDrive/LMS/rock_scissor_paper/test/paper_test/'
resize_images(image_dir_path7)

681  images to be resized.
681  images resized.


가위, 바위, 보 이미지를 28x28 사이즈로 바꾼다.

In [187]:
def load_test_data(img_path, number_of_data):
    img_size = 28
    color = 3
    
    imgs = np.zeros(number_of_data * img_size * img_size * color, dtype = np.int32).reshape(number_of_data, img_size, img_size, color)
    labels = np.zeros(number_of_data, dtype = np.int32)

    idx = 0
    for file in glob.iglob(img_path + '/scissor_test/*.jpg'):
        img = np.array(Image.open(file), dtype = np.int32)
        imgs[idx, :, :, :] = img
        labels[idx] = 0
        idx = idx + 1

    for file in glob.iglob(img_path + '/rock_test/*.jpg'):
        img = np.array(Image.open(file), dtype = np.int32)
        imgs[idx, :, :, :] = img
        labels[idx] = 1
        idx = idx + 1       
    
    for file in glob.iglob(img_path + '/paper_test/*.jpg'):
        img = np.array(Image.open(file), dtype = np.int32)
        imgs[idx, :, :, :] = img
        labels[idx] = 2
        idx = idx + 1
        
    return imgs, labels

라벨을 가위(0), 바위(1), 보(2)로 한다.
<br/>이미지와 라벨을 담은 행렬을 생성한다.
<br/>np.zeros와 reshape를 통해 행렬을 초기화한다.

In [188]:
image_dir_path = '/content/drive/MyDrive/LMS/rock_scissor_paper/test/'
(x_test, y_test)=load_test_data(image_dir_path, 2038)
x_test_norm = x_test / 255.0   

255로 나누어 입력은 0~1 사이의 값으로 정규화한다.


###Drop-Out 적용 모델 테스트

In [190]:
test1_loss, test1_accuracy = model1.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss: {} ".format(test1_loss))
print("test_accuracy: {}".format(test1_accuracy))

64/64 - 0s - loss: 0.0887 - accuracy: 0.9735 - 337ms/epoch - 5ms/step
test_loss: 0.08867946267127991 
test_accuracy: 0.9735034108161926


###Drop-Out 미적용 모델 테스트

In [191]:
test2_loss, test2_accuracy = model2.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss: {} ".format(test2_loss))
print("test_accuracy: {}".format(test2_accuracy))

64/64 - 0s - loss: 0.0903 - accuracy: 0.9794 - 379ms/epoch - 6ms/step
test_loss: 0.09028542786836624 
test_accuracy: 0.9793915748596191


###과적합 발생
<br/>훈련 데이터와 테스트 데이터가 비슷해서 모델 설정이 쉽기 때문이다.
<br/>따라서 모양이 다른 사진을 테스트 데이터로 써 본다.

##가위, 바위, 보가 너무 쉬울때!

In [192]:
image_dir_path8 = '/content/drive/MyDrive/LMS/rock_scissor_paper/real_test/scissor/'
resize_images(image_dir_path8)

4  images to be resized.
4  images resized.


In [193]:
image_dir_path9 = '/content/drive/MyDrive/LMS/rock_scissor_paper/real_test/rock/'
resize_images(image_dir_path9)

4  images to be resized.
4  images resized.


In [194]:
image_dir_path10 = '/content/drive/MyDrive/LMS/rock_scissor_paper/real_test/paper/'
resize_images(image_dir_path10)

4  images to be resized.
4  images resized.


In [195]:
def load_test2_data(img_path, number_of_data):
    img_size = 28
    color = 3
    
    imgs = np.zeros(number_of_data * img_size * img_size * color, dtype = np.int32).reshape(number_of_data, img_size, img_size, color)
    labels = np.zeros(number_of_data, dtype = np.int32)

    idx = 0
    for file in glob.iglob(img_path + '/scissor/*.jpg'):
        img = np.array(Image.open(file), dtype = np.int32)
        imgs[idx, :, :, :] = img
        labels[idx] = 0
        idx = idx + 1

    for file in glob.iglob(img_path + '/rock/*.jpg'):
        img = np.array(Image.open(file), dtype = np.int32)
        imgs[idx, :, :, :] = img
        labels[idx] = 1
        idx = idx + 1       
    
    for file in glob.iglob(img_path + '/paper/*.jpg'):
        img = np.array(Image.open(file), dtype = np.int32)
        imgs[idx, :, :, :] = img
        labels[idx] = 2
        idx = idx + 1
        
    return imgs, labels

In [196]:
image_dir_path11 = '/content/drive/MyDrive/LMS/rock_scissor_paper/real_test/'
(x_test, y_test)=load_test2_data(image_dir_path11, 12)
x_test_norm = x_test / 255.0  

###Drop-Out 적용 모델 테스트

In [198]:
test3_loss, test3_accuracy = model1.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss: {} ".format(test3_loss))
print("test_accuracy: {}".format(test3_accuracy))

1/1 - 0s - loss: 3.7691 - accuracy: 0.2500 - 21ms/epoch - 21ms/step
test_loss: 3.7690927982330322 
test_accuracy: 0.25


###Drop-Out 미적용 모델 테스트

In [199]:
test4_loss, test4_accuracy = model2.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss: {} ".format(test4_loss))
print("test_accuracy: {}".format(test4_accuracy))

1/1 - 0s - loss: 3.8002 - accuracy: 0.7500 - 20ms/epoch - 20ms/step
test_loss: 3.8001725673675537 
test_accuracy: 0.75


###과적합을 방지하는 방법

테스트 데이터의 accuracy가 75% 이하로 떨어졌다.
<br/>이유는 훈련 데이터에 대한 과적합이 발생했기 때문이다.
<br/>훈련 데이터와 다른 형태의 데이터가 들어온다면 융통성이 없어서 그 데이터에 적응을 못하는 것이다.

과적합을 방지하는 방법으로는 다음이 있다.

* 추가 데이터 수집

모형을 일반화하기 위해서는 더 많은 예제를 수집해야 한다.

* 데이터 확대 및 노이즈

데이터를 여러 가지 형태로 변형시켜 다양성에 적응할 수 있는 모델의 범용성을 확대한다.
<br/>적당한 노이즈를 추가하여 정제되지 않을 가능성 있는 데이터의 현실성을 반영한다.

* 모델 단순화

복잡한 모델을 단순화하여 모델의 학습시간을 줄인다.
<br/>이로 인해 더 많은 학습횟수를 확보할 수 있다.

##결론

'model1'과 'model2'는 과적합을 방지할 수 있는 보완점을 마련해야 한다.
<br/>그렇지 않으면 과거 1990년대 후반 ~ 2000년대 초반 MNIST 데이터셋의 매우 쉬운 학습과 같은 한계에 머무를 것이다.
<br/>이러한 MNIST의 한계를 극복하기 위해 Fashion MNIST 등 다양하고 범용성 높은 데이터셋이 사용되기 시작했다는 역사적 배경을 되돌아본다.

과적합을 방지하는 문제를 해결하고 나서야
<br/>이 프로젝트의 목표 Drop-out 하이퍼파라미터 적용이 효과가 있는지 알아보는 것을 달성할 수 있을 것이다.
<br/>현재 'model1'과 'model2' 상황으로는 비교할 수 없다.
<br/>또한 Drop-out 하이퍼파라미터는 모델 성능을 끌어올리는 최후의 수단으로 쓰일 것이라고 생각하기에 
<br/>Drop-out보다 과적합 문제 해결이 우선이다.


##참고문헌

[딥러닝 프레임워크 종류별 장. 단점 - 텐서플로, 케라스, 파이토치](https://hongong.hanbit.co.kr/%EB%94%A5%EB%9F%AC%EB%8B%9D-%ED%94%84%EB%A0%88%EC%9E%84%EC%9B%8C%ED%81%AC-%EB%B9%84%EA%B5%90-%ED%85%90%EC%84%9C%ED%94%8C%EB%A1%9C-%EC%BC%80%EB%9D%BC%EC%8A%A4-%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98/)


[Python glob.glob() 사용법](https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=siniphia&logNo=221397012627)


[사장님 몰래 하는 파이썬 업무자동화 1) Pillow 설치 및 이미지 불러오기](https://wikidocs.net/153080)


[딥러닝Drop-out(드롭아웃)은 무엇이고 왜 사용할까?](https://heytech.tistory.com/127)


[강의 01 가위 바위 보 분류 모델 학습](https://wikidocs.net/73612)

[Keras를 사용한 머신 러닝 mnist 코드 탐구 (3)](https://m.blog.naver.com/PostView.nhn?isHttpsRedirect=true&blogId=ksg97031&logNo=221302568652)


[기계학습 : 과적합을 방지하는 방법 6가지](https://iotnbigdata.tistory.com/15)